# Background

This notebook is used to convert the raw Lowland Peat Survey soil horizon information into peat thickness labels. This is non-trivial since each auger measurement consists of multiple soil horizons. Peat may be contiguous or non-contiguous and may start from a non-zero depth. Various modelling decisions are made to obtain the necessary labels.

# Algorithm

For each location I will output only a thickness, ID, northing, easting. The following algorithm is used to determine the thickness value to pass at each point:
1. No peat found at point => pass 0
2. Peat found starting at zero => pass thickness of horizon
3. Contiguous horizons of peat found => pass total thickness of combined horizons
4. Non-contiguous horizons => pass only the first (once stop seeing peat move on to next point)
5. Peat found between 0 and 50cm => exclude point
6. Peat found deeper than 50cm => pass 0 

Note: it looks like a lot of auger cores weren't taken until the end of the peat profile. Its unclear whether they were taken down until the peat stopped (e.g. 75cm or 2.5m) or whether it was just quite arbritary. Exact dates also were not provided so we have just put in the approximate year. 

# Preprocessing

In [1]:
import pandas as pd

In [3]:
# load the data in as a pandas dataframe 
df = pd.read_csv('~/Desktop/Lowland_Peat_Augers.csv')
df

,AUGERID,EASTING,NORTHING,UPPER_DEPTH,LOWER_DEPTH,HORIZON_NOTATION,TEXTURE,VON_POST,MATRIX_COLOUR,MOTTLE_ABUND,MOTTLE_COLOUR,SUB_MOTTLE_ABUND,SUB_MOTTLE_COLOUR,STONE_ABUND,STONE_SIZE,STONE_TYPE,CARBONATE,TOP_BOTTOM
0,A1-NT90/080420-01,390800,604200,0,75,NaN,organic loamy peat,NaN,5YR2/1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-calcareous,B
1,A1-NT93/680770-01,396800,637700,0,30,NaN,peat,NaN,5YR3/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-calcareous,T
2,A1-NT93/680770-01,396800,637700,30,120,NaN,peat,very strongly decomposed,5YR3/3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-calcareous,B
3,A1-NT93/700760-01,397000,637600,0,12,NaN,peat,NaN,5YR3/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-calcareous,T
4,A1-NT93/700760-01,397000,637600,12,120,NaN,peat,very strongly decomposed,5YR3/3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-calcareous,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19560,A2-TR35/410380-01,634100,153800,460,510,Oh,amorphous loamy peat,NaN,10YR2/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,calcareous,NaN
19561,A2-TR35/410380-01,634100,153800,510,700,Cg,silty clay,NaN,5G6/1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-calcareous,B
19562,A2-TR35/483414-01,634830,154140,0,30,Ap,silty clay loam,NaN,10YR3/1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,T
19563,A2-TR35/483414-01,634830,154140,30,55,Oh,amorphous peat,almost completely decomposed,10YR2/1,common,75YR4/4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# remove columns that are not of interest and remove rows with missing data
df = df[['AUGERID','EASTING','NORTHING','UPPER_DEPTH','LOWER_DEPTH','TEXTURE']]
df = df.dropna()

In [5]:
# remove any augers taken where a horizon has a lower depth < a corresponding upper depth
problem_horizons = df.loc[df['UPPER_DEPTH'] >= df['LOWER_DEPTH']]['AUGERID']
df = df[~df['AUGERID'].isin(problem_horizons.values)]
df

,AUGERID,EASTING,NORTHING,UPPER_DEPTH,LOWER_DEPTH,TEXTURE
0,A1-NT90/080420-01,390800,604200,0,75,organic loamy peat
1,A1-NT93/680770-01,396800,637700,0,30,peat
2,A1-NT93/680770-01,396800,637700,30,120,peat
3,A1-NT93/700760-01,397000,637600,0,12,peat
4,A1-NT93/700760-01,397000,637600,12,120,peat
...,...,...,...,...,...,...
19560,A2-TR35/410380-01,634100,153800,460,510,amorphous loamy peat
19561,A2-TR35/410380-01,634100,153800,510,700,silty clay
19562,A2-TR35/483414-01,634830,154140,0,30,silty clay loam
19563,A2-TR35/483414-01,634830,154140,30,55,amorphous peat


In [6]:
# convert soil types into binary 'peat' or 'other' soiltypes
def check_for_peat(soiltype):
    if 'peaty' in soiltype:
        return 'other'
    elif 'peat' in soiltype: 
        return 'peat'
    else:
        return 'other'

In [7]:
# apply the function to every row of our dataframe
df['TEXTURE'] = df.apply(lambda row : check_for_peat(row['TEXTURE']), axis = 1)
df

,AUGERID,EASTING,NORTHING,UPPER_DEPTH,LOWER_DEPTH,TEXTURE
0,A1-NT90/080420-01,390800,604200,0,75,peat
1,A1-NT93/680770-01,396800,637700,0,30,peat
2,A1-NT93/680770-01,396800,637700,30,120,peat
3,A1-NT93/700760-01,397000,637600,0,12,peat
4,A1-NT93/700760-01,397000,637600,12,120,peat
...,...,...,...,...,...,...
19560,A2-TR35/410380-01,634100,153800,460,510,peat
19561,A2-TR35/410380-01,634100,153800,510,700,other
19562,A2-TR35/483414-01,634830,154140,0,30,other
19563,A2-TR35/483414-01,634830,154140,30,55,peat


# Implement Algorithm

Now we need to implement our algorithm for deciding the depth of peat found at a given point. 

In [8]:
thickness_df = pd.DataFrame()

In [9]:
# for every horizon in the dataset
for index, horizon in df.iterrows():

    # if we have dealt with this auger already then skip to next horizon
    if (index != 0 and horizon.AUGERID in set(thickness_df['AUGERID'])):
        continue
    
    # get all horizons that apply to this auger-ID
    relevant_horizons = df.loc[df['AUGERID'] == horizon.AUGERID]

    # using our algorithm, find the thickness of peat to store for this auger
    peat_thickness = 0
    non_peat_thickness = 0
    for _, row in relevant_horizons.iterrows():
        if(row.TEXTURE == 'peat' and non_peat_thickness == 0): 
            # found more peat in contiguous column => add to thickness
            peat_thickness = row.LOWER_DEPTH

        elif(row.TEXTURE == 'peat'):
            # found some peat within 50cm of surface => exclude data point
            peat_thickness = -1
            break

        elif(peat_thickness > 0):
            # previously found peat, but found non-peat layer => exit and store thickness
            break

        else:
            # found a non-layer of peat before seeing peat
            non_peat_thickness = row.LOWER_DEPTH

            # check if had 50cm yet
            if(non_peat_thickness >= 50):
                break


    # store thickness value if we found a valid point
    if(peat_thickness >= 0):
        new_entry = pd.DataFrame([{'AUGERID': horizon.AUGERID, 
                                  'EASTING': horizon.EASTING, 
                                  'NORTHING': horizon.NORTHING, 
                                  'depth': peat_thickness}])
        thickness_df = pd.concat([thickness_df, new_entry])


In [10]:
thickness_df

,AUGERID,EASTING,NORTHING,depth
0,A1-NT90/080420-01,390800,604200,75
0,A1-NT93/680770-01,396800,637700,120
0,A1-NT93/700760-01,397000,637600,120
0,A1-NT93/700780-01,397000,637800,120
0,A1-NT93/700800-01,397000,638000,120
...,...,...,...,...
0,A2-TR35/340490-01,633400,154900,295
0,A2-TR35/360455-01,633600,154550,360
0,A2-TR35/373458-01,633730,154580,395
0,A2-TR35/378427-01,633780,154270,315


In [11]:
# save the dataframe as a csv. Will then convert coordinates in QGIS, add dates and put in our data spreadsheet.
thickness_df.to_csv("~/Desktop/LPS_processed.csv", index=False)